Global setup

In [239]:
n=4
k=2
q=41 #Lithium AAA?  #same as Alkaline D
#q should be divisible by 8 for NTT
η=1 #secret key range, prefered 2 in Alkaline
γ=2^4 #y coeff range, power of 2, exponent about 2/3 of q?  Earlier version was *not* power of 2
R.<x> = ZZ[]

In [240]:
q

41

In [241]:
is_prime(q)

True

In [242]:
qlen = ceil(log(q,2))
qlen

6

In [243]:
τ = 1 #number of 1's in c
ln(binomial(4, τ)).n()+τ #challenge entropy, should be between n/2 and n

2.38629436111989

In [244]:
β = τ*η
β

1

Chance of $\mathbf{y}_i$-coefficient passing:

In [245]:
((2*(γ-β)-1)/(2*γ-1))^(n*k).n()

0.586530272919658

Expected number of repetitions:

In [246]:
1/(((2*(γ-β)-1)/(2*γ-1))^(n*k))^2.n()

2.90682666192675

Alice generates a signing key

In [247]:
A = matrix([[randrange(0,q)*x^3+randrange(0,q)*x^2+randrange(0,q)*x+randrange(0,q), randrange(0,q)*x^3+randrange(0,q)*x^2+randrange(0,q)*x+randrange(0,q)], 
            [randrange(0,q)*x^3+randrange(0,q)*x^2+randrange(0,q)*x+randrange(0,q), randrange(0,q)*x^3+randrange(0,q)*x^2+randrange(0,q)*x+randrange(0,q)]])
A

[  17*x^3 + 34*x^2 + 4*x + 9 18*x^3 + 13*x^2 + 10*x + 38]
[ 2*x^3 + 18*x^2 + 10*x + 22   9*x^3 + 3*x^2 + 28*x + 32]

In [248]:
s1 = matrix([[randrange(-η,η+1)*x^3+randrange(-η,η+1)*x^2+randrange(-η,η+1)*x+randrange(-η,η+1)], 
            [randrange(-η,η+1)*x^3+randrange(-η,η+1)*x^2+randrange(-η,η+1)*x+randrange(-η,η+1)]])
s1

[-x^3 - x^2 + 1]
[-x^3 + x^2 + x]

In [249]:
s2 = matrix([[randrange(-η,η+1)*x^3+randrange(-η,η+1)*x^2+randrange(-η,η+1)*x+randrange(-η,η+1)], 
            [randrange(-η,η+1)*x^3+randrange(-η,η+1)*x^2+randrange(-η,η+1)*x+randrange(-η,η+1)]])
s2

[-x + 1]
[    -1]

$s_1$ and $s_2$ are private

In [250]:
t = matrix(((A*s1 + s2) % (x^n+1) % q))
t

[ -11*x^3 + 26*x^2 + 5*x + 27]
[-31*x^3 + 26*x^2 + 15*x + 24]

$A$ and $t$ are public

Alice encodes her message

In [251]:
M = 'HI_BOB__'
M_list = [Mod(ord(ch) - ord("A") + 1,q).lift_centered() for ch in M]
M_list

[8, 9, -10, 2, 15, 2, -10, -10]

Pick nonces and generate $w_1$

In [294]:
y1 = matrix([[randrange(-(γ-1), γ)*x^3+randrange(-(γ-1), γ)*x^2+randrange(-(γ-1), γ)*x+randrange(-(γ-1), γ)], 
            [randrange(-(γ-1), γ)*x^3+randrange(-(γ-1), γ)*x^2+randrange(-(γ-1), γ)*x+randrange(-(γ-1), γ)]])
y1

[12*x^3 - 8*x^2 + 9*x - 4]
[ 7*x^3 - 2*x^2 + 8*x + 7]

In [315]:
y2 = matrix([[randrange(-(γ-1), γ)*x^3+randrange(-(γ-1), γ)*x^2+randrange(-(γ-1), γ)*x+randrange(-(γ-1), γ)], 
            [randrange(-(γ-1), γ)*x^3+randrange(-(γ-1), γ)*x^2+randrange(-(γ-1), γ)*x+randrange(-(γ-1), γ)]])
y2

[ -9*x^3 + x^2 + 4*x - 1]
[-9*x^3 + 4*x^2 - x + 11]

In [296]:
w = matrix(((A*y1+y2) % (x^n+1) % q)).apply_map(lambda f: f.map_coefficients(lambda r: Mod(r,q).lift_centered() ))
w

[-19*x^3 - 8*x^2 + 15*x + 3]
[14*x^3 + 17*x^2 - 12*x - 4]

Start the "non-cryptographic hash/XOF (combined?)"

In [318]:
w_list = [c for row in w for p in row for c in p.padded_list(n)]  #flatten the matrix down to coefficients
w_list

[3, 15, -8, -19, -4, -12, 17, 14]

In [319]:
def XOF(list1, list2, itemlen, numbits):
    out = ""
    list11 = list1
    while len(out)<numbits:
        newitem = vector(list11).dot_product(vector(list2)) % (2**itemlen) # make sure all bits of input are used
        out = out + format(newitem, f'0{itemlen}b')
        list11 = [newitem] + list11[:-1]
    return out[:numbits]

In [320]:
c_hash = XOF(M_list, w_list, 6, 20)
c_hash

'11111100010100000000'

Use XOF to sample a sparse polynomial

In [321]:
clenbits = log(n,2)
clenbits

2

In [322]:
def sample_in_ball(digest):  # digest should be at least ****
    c = [0] * n
    k = n
    for i in range(n-τ, n):
        while int(digest[clenbits*k:clenbits*(k+1)], 2) > i:
            k = k+1
        j = int(digest[clenbits*k:clenbits*(k+1)], 2)
        c[i] = c[j]
        c[j] = (-1)^int(digest[i+τ-n], 2)
        k = k+1
    return c

Generate the signature


In [323]:
c = R(sample_in_ball(c_hash))
c

-x

In [324]:
z1 = matrix((y1 + c*s1) % (x^n+1))
z1

[13*x^3 - 8*x^2 + 8*x - 5]
[ 6*x^3 - 3*x^2 + 8*x + 6]

In [325]:
z2 = matrix((y2 + c*s2) % (x^n+1))
z2

[-9*x^3 + 2*x^2 + 3*x - 1]
[     -9*x^3 + 4*x^2 + 11]

Is $\|\mathbf{z}_1\|_\infty$ or $\|\mathbf{z}_2
\|_\infty \geq \gamma-\beta$?

In [326]:
γ-β

15

If so, reject and pick new $\mathbf{y}_i$

If not, $(\mathbf{z}_1, \mathbf{z}_2, c)$ is the signature

In [327]:
(z1, z2, c)

(
[13*x^3 - 8*x^2 + 8*x - 5]  [-9*x^3 + 2*x^2 + 3*x - 1]    
[ 6*x^3 - 3*x^2 + 8*x + 6], [     -9*x^3 + 4*x^2 + 11], -x
)

Bob verifies the signature

In [328]:
wprime = matrix((A*z1+z2-c*t) % (x^n+1) % q).apply_map(lambda f: f.map_coefficients(lambda r: Mod(r,q).lift_centered() ))
wprime

[  2*x^3 + 4*x^2 - 7*x - 11]
[10*x^3 - 19*x^2 - 4*x + 14]

Is $\|\mathbf{z}_i\|_\infty < \gamma-\beta$?

In [329]:
z1

[13*x^3 - 8*x^2 + 8*x - 5]
[ 6*x^3 - 3*x^2 + 8*x + 6]

In [330]:
z2

[-9*x^3 + 2*x^2 + 3*x - 1]
[     -9*x^3 + 4*x^2 + 11]

In [331]:
γ-β

15

Yes

Is $c = H(M , w_1')$?

In [332]:
wprime_list = [c for row in wprime for p in row for c in p.padded_list(n)]  #flatten the matrix down to coefficients
wprime_list

[-11, -7, 4, 2, 14, -4, -19, 10]

In [333]:
cprime_hash = XOF(M_list, wprime_list, 6, 20)
cprime_hash

'10100110101101110101'

In [334]:
cprime = R(sample_in_ball(cprime_hash))
cprime

-x^2

In [335]:
c == cprime

False

Yes.  The signature is accepted.